In [2]:
import glob
import logging
log = logging.getLogger()

# IRF

Set paths

In [3]:
fhandler = logging.FileHandler(filename='/fefs/aswg/workspace/elisa.visentin/sw_school/IRF.log', mode='a')
log.addHandler(fhandler)

scripts=('/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic/')
input_gamma=('/fefs/aswg/workspace/elisa.visentin/sw_school/gamma_2/*.h5')
output_dir_irf=('/fefs/aswg/workspace/elisa.visentin/sw_school/irf/')
config='/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml'

In [4]:
cd $scripts

/fefs/home/elisa.visentin/magic-cta-pipe/magicctapipe/scripts/lst1_magic


Get files in gammas folder and create the IRFs

In [5]:
input_file_gamma = glob.glob(input_gamma)
input_file_gamma.sort()

In [6]:
for input_file in input_file_gamma: 
    
    %run lst1_magic_create_irf.py -g $input_file -o $output_dir_irf -c $config


Quality cuts: disp_diff_mean < 0.22
Event type: software
DL2 weight type: intensity

Input gamma MC DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/gamma_2/dl2_gamma_zd_35.904deg_az_17.46deg_LST-1_MAGIC_run9502.h5

In total 677 stereo events are found:
    M1_M2 (type 0): 5 events (0.7%)
    LST1_M1 (type 1): 65 events (9.6%)
    LST1_M2 (type 2): 152 events (22.5%)
    LST1_M1_M2 (type 3): 455 events (67.2%) 

Extracting the events of the 'software' type...
--> 486 stereo events

Is diffuse MC: False

Mean true FoV offset: 0.4 deg
--> FoV offset bin: [0.3 0.5] deg

IRF type: POINT-LIKE

Is proton MC: False
Is electron MC: False

Energy bins (log space):
    start: 0.01 TeV
    stop: 1000 TeV
    n_edges: 26 

Migration bins (log space):
    start: 0.2
    stop: 5
    n_edges: 31 

Dynamic gammaness cuts:
    efficiency: 0.9
    min_cut: 0.05
    max_cut: 0.85 

Gammaness-cut table:

        low                  high         ...        cut        
        TeV             


Theta-cut table:

        low                  high         ...         cut        
        TeV                  TeV          ...         deg        
-------------------- -------------------- ... -------------------
                0.01 0.015848931924611134 ...                 0.3
0.015848931924611134 0.025118864315095794 ...                 0.3
0.025118864315095794 0.039810717055349734 ...                 0.3
0.039810717055349734  0.06309573444801933 ...                 0.3
 0.06309573444801933                  0.1 ...                 0.3
                 0.1  0.15848931924611143 ... 0.21896833224736215
                 ...                  ... ...                 ...
  39.810717055349734   63.095734448019364 ...                 0.3
  63.095734448019364                100.0 ...                 0.3
               100.0   158.48931924611142 ...                 0.3
  158.48931924611142   251.18864315095823 ...                 0.3
  251.18864315095823    398.1071705534977 ...            


Theta-cut table:

        low                  high         ...         cut        
        TeV                  TeV          ...         deg        
-------------------- -------------------- ... -------------------
                0.01 0.015848931924611134 ...                 0.3
0.015848931924611134 0.025118864315095794 ...                 0.3
0.025118864315095794 0.039810717055349734 ...                 0.3
0.039810717055349734  0.06309573444801933 ...                 0.3
 0.06309573444801933                  0.1 ...                 0.3
                 0.1  0.15848931924611143 ... 0.21273383307636548
                 ...                  ... ...                 ...
  39.810717055349734   63.095734448019364 ...                 0.3
  63.095734448019364                100.0 ...                 0.3
               100.0   158.48931924611142 ...                 0.3
  158.48931924611142   251.18864315095823 ...                 0.3
  251.18864315095823    398.1071705534977 ...            

In [7]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/IRF.log

# DL2 to DL3 data

Set paths

In [8]:
fhandler = logging.FileHandler(filename='/fefs/aswg/workspace/elisa.visentin/sw_school/DL3.log', mode='a')
log.addHandler(fhandler)

input_data=('/fefs/aswg/workspace/elisa.visentin/sw_school/real_2/*.h5')
input_dir_irf=('/fefs/aswg/workspace/elisa.visentin/sw_school/irf/')
output_dir_dl3=('/fefs/aswg/workspace/elisa.visentin/sw_school/dl3')
config=('/fefs/aswg/workspace/elisa.visentin/sw_school/config_dyn.yaml')

Get files from real data folder and update them to DL3

In [9]:
input_file_data = glob.glob(input_data)
input_file_data.sort()

In [10]:
for input_file in input_file_data:    
    %run lst1_magic_dl2_to_dl3.py -d $input_file -i $input_dir_irf -o $output_dir_dl3 -c $config


Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

The following IRF data files are found:
/fefs/aswg/workspace/elisa.visentin/sw_school/irf/irf_zd_35.904deg_az_17.462deg_software_gh_dyn0.9_theta_dyn0.75.fits.gz

Grid points in (cos(Zd), Az):

Grid points in (cos(Zd), Az):

Grid points in (cos(Zd), Az):

Grid points in (cos(Zd), Az):
array(  0.8100007  0.30476939  )
array(  0.8100007  0.30476939  )
array(  0.8100007  0.30476939  )
array(  0.8100007  0.30476939  )

Extra header:

Extra header:

Extra header:

Extra header:
    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: intensity
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    TH_EFF: 0.75
    TH_MIN: 0.


Output file: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3/dl3_LST-1_MAGIC.Run04125.0035.fits.gz

Output file: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3/dl3_LST-1_MAGIC.Run04125.0035.fits.gz

Output file: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3/dl3_LST-1_MAGIC.Run04125.0035.fits.gz

Done.

Done.

Done.

Done.

Process time: 1 [sec]


Process time: 1 [sec]


Process time: 1 [sec]


Process time: 1 [sec]


Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

Input IRF directory: /fefs/aswg/workspace/elisa.visentin/sw_school/irf/

The following IRF data files are found:
/fefs/aswg/workspace/elisa.visentin/sw_school/irf/irf_zd_35.904deg_az_17.462deg_software_gh_dyn0.9_theta_dyn0.75.fits.gz

Grid points in (cos(Zd), Az):

Grid points


Gammaness cut table:

        low                  high                cut        
        TeV                  TeV                            
-------------------- -------------------- ------------------
                0.01 0.015848931924611134               0.05
0.015848931924611134 0.025118864315095794               0.05
0.025118864315095794 0.039810717055349734               0.05
0.039810717055349734  0.06309573444801933               0.05
 0.06309573444801933                  0.1               0.05
                 0.1  0.15848931924611143 0.5601413857858907
                 ...                  ...                ...
  39.810717055349734   63.095734448019364               0.05
  63.095734448019364                100.0               0.05
               100.0   158.48931924611142               0.05
  158.48931924611142   251.18864315095823               0.05
  251.18864315095823    398.1071705534977               0.05
   398.1071705534977    630.9573444801943               0.05
 


Gammaness cut table:

        low                  high                cut        
        TeV                  TeV                            
-------------------- -------------------- ------------------
                0.01 0.015848931924611134               0.05
0.015848931924611134 0.025118864315095794               0.05
0.025118864315095794 0.039810717055349734               0.05
0.039810717055349734  0.06309573444801933               0.05
 0.06309573444801933                  0.1               0.05
                 0.1  0.15848931924611143 0.5601413857858907
                 ...                  ...                ...
  39.810717055349734   63.095734448019364               0.05
  63.095734448019364                100.0               0.05
               100.0   158.48931924611142               0.05
  158.48931924611142   251.18864315095823               0.05
  251.18864315095823    398.1071705534977               0.05
   398.1071705534977    630.9573444801943               0.05
 

    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: intensity
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    TH_EFF: 0.75
    TH_MIN: 0.1
    TH_MAX: 0.3 
    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: intensity
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    TH_EFF: 0.75
    TH_MIN: 0.1
    TH_MAX: 0.3 
    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: intensity
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    TH_EFF: 0.75
    TH_MIN: 0.1
    TH_MAX: 0.3 
    TELESCOP: CTA-N
    INSTRUME: LST-1_MAGIC
    FOVALIGN: RADEC
    QUAL_CUT: disp_diff_mean < 0.22
    EVT_TYPE: software
    DL2_WEIG: intensity
    GH_EFF: 0.9
    GH_MIN: 0.05
    GH_MAX: 0.85
    TH_EFF: 0.75
    TH_MIN: 0.1
    TH_MAX: 0.3 
    


Gammaness cut table:

        low                  high                cut        
        TeV                  TeV                            
-------------------- -------------------- ------------------
                0.01 0.015848931924611134               0.05
0.015848931924611134 0.025118864315095794               0.05
0.025118864315095794 0.039810717055349734               0.05
0.039810717055349734  0.06309573444801933               0.05
 0.06309573444801933                  0.1               0.05
                 0.1  0.15848931924611143 0.5601413857858907
                 ...                  ...                ...
  39.810717055349734   63.095734448019364               0.05
  63.095734448019364                100.0               0.05
               100.0   158.48931924611142               0.05
  158.48931924611142   251.18864315095823               0.05
  251.18864315095823    398.1071705534977               0.05
   398.1071705534977    630.9573444801943               0.05
 


Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

Input DL2 data file: /fefs/aswg/workspace/elisa.visentin/sw_school/real_2/dl2_LST-1_MAGIC.Run04125.0039.h5

In total 170 stereo events are found:
    M1_M2 (type 0): 31 events (18.2%)
    LST1_M1 (type 1): 8 events (4.7%)
    LST1_M2 (type 2): 51 events (30.0%)
    LST1_M1_M2 (type 3): 80 events (47.1%) 

Extracting the events of the 'software' type...
--> 139 stereo events

Calculating the dead time correction factor...
LST-1: 0.962
MAGIC(-II): 0.994
-->


Gammaness cut table:

        low                  high                cut        
        TeV                  TeV                            
-------------------- -------------------- ------------------
                0.01 0.015848931924611134               0.05
0.015848931924611134 0.025118864315095794               0.05
0.025118864315095794 0.039810717055349734               0.05
0.039810717055349734  0.06309573444801933               0.05
 0.06309573444801933                  0.1               0.05
                 0.1  0.15848931924611143 0.5601413857858907
                 ...                  ...                ...
  39.810717055349734   63.095734448019364               0.05
  63.095734448019364                100.0               0.05
               100.0   158.48931924611142               0.05
  158.48931924611142   251.18864315095823               0.05
  251.18864315095823    398.1071705534977               0.05
   398.1071705534977    630.9573444801943               0.05
 

Create HDU and observation index files

In [11]:
%run create_dl3_index_files.py -i $output_dir_dl3


Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

Input directory: /fefs/aswg/workspace/elisa.visentin/sw_school/dl3

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:

The following DL3 data files are found:
/fefs/aswg/workspac

In [12]:
more /fefs/aswg/workspace/elisa.visentin/sw_school/DL3.log